In [30]:
import pandas as pd
import requests
from datetime import datetime, timedelta
import numpy as np
from get7DMA import get_7dma_state

In [31]:
def col_check_state_raw_csv(df):
    cols_ls=df.columns
    reqd_col_ls=["Date",
                 "State/UTCode",
                 "deltaConfirmedForState",
                 "deltaDeceasedForState",
                 "deltaRecoveredForState",
                 "deltaTestedForState",
                 "deltaVaccinated1ForState",
                 "deltaVaccinated2ForState",
                 "delta21_14confirmedForState",
                 "7DmaConfirmedForState",
                 "7DmaDeceasedForState",
                 "7DmaRecoveredForState",
                 "7DmaTestedForState",
                 "7DmaVaccinated1ForState",
                 "7DmaVaccinated2ForState",
                 "District",
                 "deltaConfirmedForDistrict",
                 "deltaDeceasedForDistrict",
                 "deltaRecoveredForDistrict",
                 "deltaTestedForDistrict",
                 "deltaVaccinated1ForDistrict",
                 "deltaVaccinated2ForDistrict",
                 "delta21_14confirmedForDistrict",
                 "7DmaConfirmedForDistrict",
                 "7DmaDeceasedForDistrict",
                 "7DmaRecoveredForDistrict",
                 "7DmaTestedForDistrict",
                 "7DmaVaccinated1ForDistrict",
                 "7DmaVaccinated2ForDistrict",
                 "districtPopulation",
                 "tested_last_updated_district",
                 "tested_source_district",
                 "notesForDistrict",
                 "cumulativeConfirmedNumberForDistrict",
                 "cumulativeDeceasedNumberForDistrict",
                 "cumulativeRecoveredNumberForDistrict",
                 "cumulativeTestedNumberForDistrict",
                 "cumulativeVaccinated1NumberForDistrict",
                 "cumulativeVaccinated2NumberForDistrict",
                 "last_updated",
                 "statePopulation",
                 "tested_last_updated_state",
                 "tested_source_state",
                 "notesForState",
                 "cumulativeConfirmedNumberForState",
                 "cumulativeDeceasedNumberForState",
                 "cumulativeRecoveredNumberForState",
                 "cumulativeTestedNumberForState",
                 "cumulativeVaccinated1NumberForState",
                 "cumulativeVaccinated2NumberForState"
                ]
    for col in reqd_col_ls:
        if col not in cols_ls:
            df[col]=0
    df_formatted=df[reqd_col_ls]
    return df_formatted
            

In [32]:
#https://github.com/covid19india/covid19india-react/blob/master/src/constants.js#L398
STATE_NAMES = {
  'AP': 'Andhra Pradesh',
  'AR': 'Arunachal Pradesh',
  'AS': 'Assam',
  'BR': 'Bihar',
  'CT': 'Chhattisgarh',
  'GA': 'Goa',
  'GJ': 'Gujarat',
  'HR': 'Haryana',
  'HP': 'Himachal Pradesh',
  'JH': 'Jharkhand',
  'KA': 'Karnataka',
  'KL': 'Kerala',
  'MP': 'Madhya Pradesh',
  'MH': 'Maharashtra',
  'MN': 'Manipur',
  'ML': 'Meghalaya',
  'MZ': 'Mizoram',
  'NL': 'Nagaland',
  'OR': 'Odisha',
  'PB': 'Punjab',
  'RJ': 'Rajasthan',
  'SK': 'Sikkim',
  'TN': 'Tamil Nadu',
  'TG': 'Telangana',
  'TR': 'Tripura',
  'UT': 'Uttarakhand',
  'UP': 'Uttar Pradesh',
  'WB': 'West Bengal',
  'AN': 'Andaman and Nicobar Islands',
  'CH': 'Chandigarh',
  'DN': 'Dadra and Nagar Haveli and Daman and Diu',
  'DL': 'Delhi',
  'JK': 'Jammu and Kashmir',
  'LA': 'Ladakh',
  'LD': 'Lakshadweep',
  'PY': 'Puducherry',
  # 'TT': 'India',
 # [UNASSIGNED_STATE_CODE]: 'Unassigned',
}

In [33]:
# cowin_codes=pd.read_csv("../CSV/CowinAppStateAndDistrictCode.csv")
# cowin_codes.head()

In [34]:
#Creating the mapping dictionary for multiple districts addition scenario
cowin_codes=pd.read_csv("../CSV/CowinAppStateAndDistrictCode.csv")
cowin_codes_map={}
cowin_codes["district_id_add"]=cowin_codes[["district_id_add"]].fillna("NULL")

for i,d_id in enumerate(cowin_codes["district_id"]):
#     print(d_id)
    add_dist_id=cowin_codes.loc[i,"district_id_add"]
    if add_dist_id !="NULL":
        cowin_codes_map[str(int(d_id))]=[str(int(add_dist_id)),str(int(d_id))]
    else:
        cowin_codes_map[str(int(d_id))]=[str(int(d_id))]
# print(cowin_codes_map)

In [35]:
cowin_codes_map

{'3': ['3'],
 '1': ['1'],
 '2': ['2'],
 '9': ['9'],
 '10': ['10'],
 '11': ['11'],
 '5': ['5'],
 '4': ['4'],
 '7': ['7'],
 '12': ['12'],
 '13': ['13'],
 '14': ['14'],
 '8': ['8'],
 '15': ['15'],
 '16': ['16'],
 '6': ['6'],
 '22': ['22'],
 '20': ['20'],
 '25': ['25'],
 '23': ['23'],
 '42': ['42'],
 '17': ['39', '17'],
 '24': ['24'],
 '27': ['27'],
 '21': ['21'],
 '33': ['33'],
 '29': ['29'],
 '40': ['40'],
 '31': ['31'],
 '18': ['18'],
 '32': ['32'],
 '36': ['36'],
 '19': ['19'],
 '39': ['17', '39'],
 '35': ['35'],
 '37': ['37'],
 '30': ['30'],
 '26': ['26'],
 '34': ['34'],
 '41': ['41'],
 '28': ['28'],
 '38': ['38'],
 '46': ['46'],
 '47': ['47'],
 '765': ['765'],
 '57': ['57'],
 '66': ['66'],
 '766': ['766'],
 '58': ['58'],
 '48': ['48'],
 '62': ['62'],
 '59': ['59'],
 '43': ['43'],
 '67': ['67'],
 '60': ['60'],
 '53': ['53'],
 '68': ['68'],
 '764': ['764'],
 '54': ['54'],
 '49': ['49'],
 '50': ['50'],
 '51': ['51'],
 '69': ['69'],
 '61': ['61'],
 '63': ['63'],
 '767': ['767'],
 '55': [

In [36]:
def vaccination_numbers_api(state_name,Date):
    cowin_codes=pd.read_csv("../CSV/CowinAppStateAndDistrictCode.csv")
    
    if state_name=="Dadra and Nagar Haveli and Daman and Diu":
        vaccination_number_DNH=vaccination_numbers_api("Dadra and Nagar Haveli",Date)
        vaccination_number_DND=vaccination_numbers_api("Daman and Diu",Date)
        
        state_cumulative_vaccinated1=vaccination_number_DNH[0]+vaccination_number_DND[0]
        state_cumulative_vaccinated2=vaccination_number_DNH[1]+vaccination_number_DND[1]
        state_cumulative_vaccinated=vaccination_number_DNH[2]+vaccination_number_DND[2]
        district_vacc_dict=vaccination_number_DNH[3]
        district_vacc_dict.update(vaccination_number_DND[3])
        
        return(state_cumulative_vaccinated1,state_cumulative_vaccinated2,state_cumulative_vaccinated,district_vacc_dict)
    
#     print(state_name)
    district_cumulative_vaccinated1=0
    district_cumulative_vaccinated2=0
    district_cumulative_vaccinated=0
    
    state_cumulative_vaccinated1=0
    state_cumulative_vaccinated2=0
    state_cumulative_vaccinated=0
        
    #State Numbers
    #State Numbers
    if state_name=="Dadra and Nagar Haveli":
        cowin_state_code=8
    elif state_name=="Daman and Diu":
        cowin_state_code=37
    else:
        cowin_state_code=cowin_codes.loc[cowin_codes["state_name"]==state_name,"state_id"]
        cowin_state_code.reset_index(inplace=True,drop=True)
        cowin_state_code=list(set(cowin_state_code))[0]
    
    
    api_url_state="https://api.cowin.gov.in/api/v1/reports/v2/getPublicReports?state_id="+str(cowin_state_code)+"&date="+Date
#     print(api_url_state)
    api_data_state=requests.get(api_url_state)
#     print(api_data_state.json())
    
    state_cumulative_vaccinated1=api_data_state.json()["topBlock"]["vaccination"]["tot_dose_1"]
    state_cumulative_vaccinated2=api_data_state.json()["topBlock"]["vaccination"]["tot_dose_2"]
    state_cumulative_vaccinated=api_data_state.json()["topBlock"]["vaccination"]["total"]
    
    district_vacc_dict={}
    for ele in api_data_state.json()["getBeneficiariesGroupBy"]:
        district_vacc_dict[ele["district_id"]]=[ele["partial_vaccinated"],ele["totally_vaccinated"],ele["total"]]
    
    return (state_cumulative_vaccinated1,state_cumulative_vaccinated2,state_cumulative_vaccinated,district_vacc_dict)

In [37]:
vaccination_numbers_api("Karnataka","2021-12-02")[3]

{'264': [3409823, 2125192, 5535015],
 '265': [1275123, 896678, 2171801],
 '266': [2271321, 1638462, 3909783],
 '267': [1621608, 922777, 2544385],
 '268': [1163787, 794547, 1958334],
 '269': [1566903, 1132212, 2699115],
 '270': [1270240, 841156, 2111396],
 '271': [676204, 492646, 1168850],
 '272': [1029998, 699584, 1729582],
 '273': [756141, 483154, 1239295],
 '274': [1937307, 1159920, 3097227],
 '275': [1135839, 695500, 1831339],
 '276': [728318, 531519, 1259837],
 '277': [1083277, 740126, 1823403],
 '278': [1325817, 840987, 2166804],
 '279': [1056225, 632563, 1688788],
 '280': [779436, 461921, 1241357],
 '281': [1015318, 726832, 1742150],
 '282': [878531, 586684, 1465215],
 '283': [393364, 307604, 700968],
 '284': [1196815, 710099, 1906914],
 '285': [750746, 440577, 1191323],
 '286': [939549, 726137, 1665686],
 '287': [1205374, 761104, 1966478],
 '288': [1850463, 1322530, 3172993],
 '289': [1262608, 829768, 2092376],
 '290': [1239853, 967632, 2207485],
 '291': [896090, 612094, 1508184

In [38]:
def updateDerivedValues(StateCode,Date):
    cowin_codes=pd.read_csv("../CSV/CowinAppStateAndDistrictCode.csv")
    try:
        state_raw_csv=pd.read_csv(f"../RAWCSV/{Date}/{StateCode}_raw.csv",index_col=False)
    except:
        state_raw_csv=pd.read_csv(f"../RAWCSV/{Date}/myGov/{StateCode}_raw.csv",index_col=False)
        # RAWCSV/2021-11-05/myGov/AN_raw.csv
    state_population=pd.read_csv("../CSV/StatePopulation.csv")
    district_population=pd.read_csv("../CSV/DistrictPopulation.csv")
    
    #Check and insert cummulativeDeceasedState 
    # if state_raw_csv["cumulativeDeceasedNumberForState"].sum() == 0:
    #     if state_raw_csv["cumulativeDeceasedNumberForDistrict"].sum() != 0:
    #         state_raw_csv["cumulativeDeceasedNumberForState"] = state_raw_csv["cumulativeDeceasedNumberForDistrict"].sum()
    state_raw_csv=col_check_state_raw_csv(state_raw_csv)
    
    state_name=STATE_NAMES[StateCode]
    
    print("\n")
    print(state_name)
    print("---------------------------------")
    
    state_raw_csv["State/UTCode"]=StateCode
    
    # if StateCode == "DN":
    #     return None
    # #gunjan will provide
    # else:
    vaccination_numbers=vaccination_numbers_api(state_name,Date)
    
    state_cumulative_vaccinated1=vaccination_numbers[0]
    state_cumulative_vaccinated2=vaccination_numbers[1]
    state_cumulative_vaccinated=vaccination_numbers[2]
    
    state_raw_csv.loc[state_raw_csv["State/UTCode"]==StateCode,"cumulativeVaccinated1NumberForState"]=state_cumulative_vaccinated1
    state_raw_csv.loc[state_raw_csv["State/UTCode"]==StateCode,"cumulativeVaccinated2NumberForState"]=state_cumulative_vaccinated2
    state_raw_csv.loc[state_raw_csv["State/UTCode"]==StateCode,"cumulativeVaccinatedNumberForState"]=state_cumulative_vaccinated
    
    #Reading State population numbers from static source
    state_population_number=state_population.loc[state_population["State"]==StateCode,"StatePop"]
    state_population_number.reset_index(inplace=True,drop=True)
    state_population_number=state_population_number[0]    
    state_raw_csv.loc[state_raw_csv["State/UTCode"]==StateCode,"statePopulation"]=state_population_number
    
    #Deriving the delta numbers
    
    prev_date=datetime.strptime(Date,"%Y-%m-%d")+timedelta(days=(-1))
    prev_date_str=str(prev_date.date())
    try:
        previous_state_raw_csv=pd.read_csv(f"../RAWCSV/{prev_date_str}/{StateCode}_raw.csv")
    except:
        previous_state_raw_csv=pd.read_csv(f"../RAWCSV/{prev_date_str}/myGov/{StateCode}_raw.csv")
    
    vaccination_numbers_yesterday=vaccination_numbers_api(state_name,prev_date_str)
    
    try:
        if state_raw_csv["cumulativeConfirmedNumberForState"].sum() != 0:
            state_raw_csv["deltaConfirmedForState"]=state_raw_csv["cumulativeConfirmedNumberForState"]-previous_state_raw_csv["cumulativeConfirmedNumberForState"]
    except:
        state_raw_csv["deltaConfirmedForState"] = None
    
    try:
        if state_raw_csv["cumulativeDeceasedNumberForState"].sum() != 0:
            state_raw_csv["deltaDeceasedForState"]=state_raw_csv["cumulativeDeceasedNumberForState"]-previous_state_raw_csv["cumulativeDeceasedNumberForState"]
    except:
        state_raw_csv["deltaDeceasedForState"] = None
        
    try:    
        if state_raw_csv["cumulativeRecoveredNumberForState"].sum() != 0:
            state_raw_csv["deltaRecoveredForState"]=state_raw_csv["cumulativeRecoveredNumberForState"]-previous_state_raw_csv["cumulativeRecoveredNumberForState"]
    except:
        state_raw_csv["deltaDeceasedForState"] = None
    
    try:
        if state_raw_csv["cumulativeVaccinatedNumberForState"].sum() != 0:
            state_raw_csv["deltaVaccinatedForState"]=state_raw_csv["cumulativeVaccinatedNumberForState"][0]-vaccination_numbers_yesterday[2]
    except:
        state_raw_csv["deltaVaccinatedForState"] = None
        
    try:    
        if state_raw_csv["cumulativeVaccinated1NumberForState"].sum() != 0:
            state_raw_csv["deltaVaccinated1ForState"]=state_raw_csv["cumulativeVaccinated1NumberForState"]-vaccination_numbers_yesterday[0]
    except:
        state_raw_csv["deltaVaccinated1ForState"] =None
    
    try:
        if state_raw_csv["cumulativeVaccinated2NumberForState"].sum() != 0:
            state_raw_csv["deltaVaccinated2ForState"]=state_raw_csv["cumulativeVaccinated2NumberForState"]-vaccination_numbers_yesterday[1]
    except:
        state_raw_csv["deltaVaccinated2ForState"] = None
    
    try:        
        if state_raw_csv["cumulativeTestedNumberForState"].sum() != 0:
            state_raw_csv["deltaTestedForState"]=state_raw_csv["cumulativeTestedNumberForState"]-previous_state_raw_csv["cumulativeTestedNumberForState"]
    except FileNotFoundError:
        state_raw_csv["deltaTestedForState"] = None
        
#     state_raw_csv.loc[:,"7DmaConfirmedForState"]=0
#     state_raw_csv.loc[:,"7DmaDeceasedForState"]=0
#     state_raw_csv.loc[:,"7DmaRecoveredForState"]=0
#     state_raw_csv.loc[:,"7DmaVaccinatedForState"]=0
#     state_raw_csv.loc[:,"7DmaTestedForState"]=0
#     state_raw_csv.loc[:,"7DmaVaccinated1ForState"]=0
#     state_raw_csv.loc[:,"7DmaVaccinated2ForState"]=0
    
#     try:
#         avg_counter = 0
#         for i in range(7):
#             prev_date=datetime.strptime(Date,"%Y-%m-%d")+timedelta(days=(-1*i))
#             # print(str(prev_date))
#             prev_date_str=str(prev_date.date())
#             if i == 0:
#                 previous_state_raw_csv = state_raw_csv
#             else:
#                 try:
#                     previous_state_raw_csv=pd.read_csv(f"../RAWCSV/{prev_date_str}/{StateCode}_raw.csv")
#                 except:
#                     previous_state_raw_csv=pd.read_csv(f"../RAWCSV/{prev_date_str}/myGov/{StateCode}_raw.csv")
                
#             #state_raw_csv["7DmaConfirmedForState"]+=int(previous_state_raw_csv["cumulativeConfirmedNumberForState"][0])
#             #state_raw_csv["7DmaDeceasedForState"]+=int(previous_state_raw_csv["cumulativeDeceasedNumberForState"][0])
#             state_raw_csv["7DmaConfirmedForState"]+=previous_state_raw_csv["cumulativeConfirmedNumberForState"][0]
#             state_raw_csv["7DmaDeceasedForState"]+=previous_state_raw_csv["cumulativeDeceasedNumberForState"][0]
#             state_raw_csv["7DmaRecoveredForState"]+=previous_state_raw_csv["cumulativeRecoveredNumberForState"][0]
#             state_raw_csv["7DmaVaccinatedForState"]+=vaccination_numbers[2]
#             state_raw_csv["7DmaTestedForState"]+=previous_state_raw_csv["cumulativeTestedNumberForState"][0]
#             state_raw_csv["7DmaVaccinated1ForState"]+=vaccination_numbers[0]
#             state_raw_csv["7DmaVaccinated2ForState"]+=vaccination_numbers[1]
#             avg_counter += 1

            
#         state_raw_csv["7DmaConfirmedForState"]=(state_raw_csv["7DmaConfirmedForState"]/7)#.astype('int64')
#         state_raw_csv["7DmaDeceasedForState"]=(state_raw_csv["7DmaDeceasedForState"]/7)#.astype('int64')
#         state_raw_csv["7DmaRecoveredForState"]=round(state_raw_csv["7DmaRecoveredForState"]/7,0)
#         state_raw_csv["7DmaVaccinatedForState"]=round(state_raw_csv["7DmaVaccinatedForState"]/7,0)
#         state_raw_csv["7DmaTestedForState"]=round(state_raw_csv["7DmaTestedForState"]/7,0)
#         state_raw_csv["7DmaVaccinated1ForState"]=round(state_raw_csv["7DmaVaccinated1ForState"]/7,0)
#         state_raw_csv["7DmaVaccinated2ForState"]=round(state_raw_csv["7DmaVaccinated2ForState"]/7,0)
#     except:
#         raise
#         print("Error:"+state_name)
        
        
        
    for district in state_raw_csv["District"]:
        # print(district,state_name)
        cowin_district_code=cowin_codes.loc[((cowin_codes["district_name"]==district)&(cowin_codes["state_name"]==state_name)),"district_id"]
        cowin_district_code.reset_index(inplace=True,drop=True)
        try:
            cowin_district_code=cowin_district_code[0]
        except IndexError:
            # print(f"{district} not found in Cowin States Districts Code")
            district_vaccination_data=False
            pass
        except KeyError:
            # print(f"{district} not found in Cowin States Districts Code")
            district_vaccination_data=False
            pass
        else:
            ls_districts_add=cowin_codes_map[str(cowin_district_code)]
            district_cumulative_vaccinated1=0
            district_cumulative_vaccinated2=0
            district_cumulative_vaccinated=0
            for dist in ls_districts_add:
                district_cumulative_vaccinated1+=vaccination_numbers[3][dist][0]
                district_cumulative_vaccinated2+=vaccination_numbers[3][dist][1]
                district_cumulative_vaccinated+=vaccination_numbers[3][dist][2]
                district_vaccination_data=True
                
        if district_vaccination_data:
            state_raw_csv.loc[((state_raw_csv["State/UTCode"]==StateCode)&(state_raw_csv["District"]==district)),"cumulativeVaccinated1NumberForDistrict"]=district_cumulative_vaccinated1
            state_raw_csv.loc[((state_raw_csv["State/UTCode"]==StateCode)&(state_raw_csv["District"]==district)),"cumulativeVaccinated2NumberForDistrict"]=district_cumulative_vaccinated2
            state_raw_csv.loc[((state_raw_csv["State/UTCode"]==StateCode)&(state_raw_csv["District"]==district)),"cumulativeVaccinatedNumberForDistrict"]=district_cumulative_vaccinated

        
        district_population_number=district_population.loc[((district_population["State"]==StateCode) & (district_population["District"]==district)) ,"DistrictPop"]#[0]
        district_population_number.reset_index(inplace=True,drop=True)
        try:
            district_population_number=district_population_number[0]
        except IndexError:
            print(f"{district} population numbers not found")
            pass
        except KeyError:
            print(f"{district} population numbers not found")
            pass
        else:
            state_raw_csv.loc[((state_raw_csv["State/UTCode"]==StateCode)&(state_raw_csv["District"]==district)),"districtPopulation"]=district_population_number
    
    #delta for district
#         prev_date=datetime.strptime(Date,"%Y-%m-%d")+timedelta(days=(-1))
#         prev_date_str=str(prev_date.year)+"-"+str(prev_date.month)+"-"+str(prev_date.day)
#         previous_state_raw_csv=pd.read_csv(f"../RAWCSV/{prev_date_str}/{StateCode}_raw.csv")
        
#         if state_raw_csv["cumulativeConfirmedNumberForDistrict"].sum() != 0:
#             state_raw_csv.loc[state_raw_csv["District"] == district,"deltaConfirmedForDistrict"]=state_raw_csv.loc[state_raw_csv["District"] == district,"cumulativeConfirmedNumberForDistrict"].item()-previous_state_raw_csv.loc[previous_state_raw_csv["District"] == district,"cumulativeConfirmedNumberForDistrict"].item()

#         if state_raw_csv["cumulativeDeceasedNumberForDistrict"].sum() != 0:
#             state_raw_csv.loc[state_raw_csv["District"] == district,"deltaDeceasedForDistrict"]=state_raw_csv.loc[state_raw_csv["District"] == district,"cumulativeDeceasedNumberForDistrict"].item()-previous_state_raw_csv.loc[previous_state_raw_csv["District"] == district,"cumulativeDeceasedNumberForDistrict"].item()

#         if state_raw_csv["cumulativeRecoveredNumberForDistrict"].sum() != 0:
#             state_raw_csv.loc[state_raw_csv["District"] == district,"deltaRecoveredForDistrict"]=state_raw_csv.loc[state_raw_csv["District"] == district,"cumulativeRecoveredNumberForDistrict"].item()-previous_state_raw_csv.loc[previous_state_raw_csv["District"]  == district,"cumulativeRecoveredNumberForDistrict"].item()

#         if state_raw_csv["cumulativeTestedNumberForDistrict"].sum() != 0:
#             state_raw_csv.loc[state_raw_csv["District"] == district,"deltaTestedForDistrict"]=state_raw_csv.loc[state_raw_csv["District"] == district,"cumulativeTestedNumberForDistrict"].item()-previous_state_raw_csv.loc[previous_state_raw_csv["District"] == district,"cumulativeTestedNumberForDistrict"].item()

#         if state_raw_csv["cumulativeVaccinatedNumberForDistrict"].sum() != 0:
#             state_raw_csv.loc[state_raw_csv["District"] == district,"deltaVaccinatedForDistrict"]=state_raw_csv.loc[state_raw_csv["District"] == district,"cumulativeVaccinatedNumberForDistrict"].item()-previous_state_raw_csv.loc[previous_state_raw_csv["District"] == district,"cumulativeVaccinatedNumberForDistrict"].item()

#         if state_raw_csv["cumulativeVaccinated1NumberForDistrict"].sum() != 0:
#             state_raw_csv.loc[state_raw_csv["District"] == district,"deltaVaccinated1ForDistrict"]=state_raw_csv.loc[state_raw_csv["District"] == district,"cumulativeVaccinated1NumberForDistrict"].item()-previous_state_raw_csv.loc[previous_state_raw_csv["District"] == district,"cumulativeVaccinated1NumberForDistrict"].item()

#         if state_raw_csv["cumulativeVaccinated2NumberForDistrict"].sum() != 0:
#             state_raw_csv.loc[state_raw_csv["District"] == district,"deltaVaccinated2ForDistrict"]=state_raw_csv.loc[state_raw_csv["District"] == district,"cumulativeVaccinated2NumberForDistrict"].item()-previous_state_raw_csv.loc[previous_state_raw_csv["District"] == district,"cumulativeVaccinated2NumberForDistrict"].item()
           
    lst = ["Recovered","Confirmed","Deceased","Tested","Vaccinated1","Vaccinated2","Vaccinated"]
    for val in lst:
        state_raw_csv["delta{}ForDistrict".format(val)] = 0
        for idx in state_raw_csv.index:
            # print(new_state_raw_csv["District"][idx])
            # print(new_state_raw_csv["7DmaVaccinatedForDistrict"][idx])
            avg_counter = 0
            value = 0
            i = 1
            prev_date=datetime.strptime(Date,"%Y-%m-%d")+timedelta(days=(-1*i))
            prev_date_str=str(prev_date.date())
            if i == 0:
                previous_state_raw_csv = state_raw_csv
            else:
                previous_state_raw_csv_vacc=pd.read_csv(f"../RAWCSV/{prev_date_str}/{StateCode}_final.csv")
                try:
                    previous_state_raw_csv=pd.read_csv(f"../RAWCSV/{prev_date_str}/{StateCode}_raw.csv")
                except:
                    previous_state_raw_csv=pd.read_csv(f"../RAWCSV/{prev_date_str}/myGov/{StateCode}_raw.csv")
                
            cowin_district_code=cowin_codes.loc[((cowin_codes["district_name"]==state_raw_csv["District"][idx])&(cowin_codes["state_name"]==state_name)),"district_id"]
            cowin_district_code.reset_index(inplace=True,drop=True)
            try:
                cowin_district_code=cowin_district_code[0]
                district_vaccination_data=True
            except IndexError:
                # print(f"{district} not found in Cowin States Districts Code")
                district_vaccination_data=False
                pass
            except KeyError:
                # print(f"{district} not found in Cowin States Districts Code")
                district_vaccination_data=False
                pass
            #Read final file for vaccination
            
            if val in ["Vaccinated1","Vaccinated2"]:
                # if district_vaccination_data:
                #     value = state_raw_csv["cumulative{}NumberForDistrict".format(val)][idx] - vaccination_numbers_yesterday[3][str(cowin_district_code)][["Vaccinated1","Vaccinated2","Vaccinated"].index(val)]
                
                value = state_raw_csv["cumulative{}NumberForDistrict".format(val)][idx] - previous_state_raw_csv_vacc.loc[previous_state_raw_csv_vacc["District"] == state_raw_csv["District"][idx],"cumulative{}NumberForDistrict".format(val)]
            elif val != "Vaccinated":    
                value = state_raw_csv["cumulative{}NumberForDistrict".format(val)][idx] - previous_state_raw_csv.loc[previous_state_raw_csv["District"] == state_raw_csv["District"][idx],"cumulative{}NumberForDistrict".format(val)]
            try:
                state_raw_csv["delta{}ForDistrict".format(val)][idx] = value
            except:
                state_raw_csv["delta{}ForDistrict".format(val)][idx] = None
    
    lst = ["Recovered","Confirmed","Deceased","Tested","Vaccinated1","Vaccinated2"]
#     for val in lst:
#         state_raw_csv["7Dma{}ForDistrict".format(val)] = 0
#         for idx in state_raw_csv.index:
#             # print(new_state_raw_csv["District"][idx])
#             # print(new_state_raw_csv["7DmaVaccinatedForDistrict"][idx])
#             avg_counter = 0
#             value = 0
#             for i in range(7):
#                 prev_date=datetime.strptime(Date,"%Y-%m-%d")+timedelta(days=(-1*i))
#                 prev_date_str=str(prev_date.date())
#                 if i == 0:
#                     previous_state_raw_csv = state_raw_csv
#                 else:
#                     try:
#                         previous_state_raw_csv=pd.read_csv(f"../RAWCSV/{prev_date_str}/{StateCode}_raw.csv")
#                     except:
#                         previous_state_raw_csv=pd.read_csv(f"../RAWCSV/{prev_date_str}/myGov/{StateCode}_raw.csv")
                
#                 cowin_district_code=cowin_codes.loc[((cowin_codes["district_name"]==state_raw_csv["District"][idx])&(cowin_codes["state_name"]==state_name)),"district_id"]
#                 cowin_district_code.reset_index(inplace=True,drop=True)
#                 try:
#                     cowin_district_code=cowin_district_code[0]
#                     district_vaccination_data=True
#                 except IndexError:
#                     # print(f"{district} not found in Cowin States Districts Code")
#                     district_vaccination_data=False
#                     pass
#                 except KeyError:
#                     # print(f"{district} not found in Cowin States Districts Code")
#                     district_vaccination_data=False
#                     pass
#                 else:
#                     ls_districts_add=cowin_codes_map[str(cowin_district_code)]

#                 if val in ["Vaccinated1","Vaccinated2","Vaccinated"]:
#                     if district_vaccination_data:
#                         value += vaccination_numbers[3][str(cowin_district_code)][["Vaccinated1","Vaccinated2","Vaccinated"].index(val)]
#                 else:    
#                     value += previous_state_raw_csv.loc[previous_state_raw_csv["District"] == state_raw_csv["District"][idx],"cumulative{}NumberForDistrict".format(val)]
#                 avg_counter += 1
#             try:
#                 state_raw_csv["7Dma{}ForDistrict".format(val)][idx] = value/7
#             except:
#                 state_raw_csv["7Dma{}ForDistrict".format(val)][idx] = None

    state_raw_csv.to_csv(f"../RAWCSV/{Date}/{StateCode}_final.csv",index=False)
    print (f"Running 7DMA for {StateCode}")
    get_7dma_state(StateCode, Date)
    
        # return(state_raw_csv)

In [39]:
# updateDerivedValues("TN","2021-12-04")
2+2

4

# Run Block

In [40]:
# import pandas as pd
# sources = pd.read_csv("../sources.csv")
# # updateDerivedValues("AS","2021-10-29")
# sources

for key,val in STATE_NAMES.items():
    updateDerivedValues(key,"2021-12-08")
    
# 7DMA is being called for "TT" separately since TT.py 7DMA calculations are not correct 
get_7dma_state('TT', '2021-12-08')
    



Andhra Pradesh
---------------------------------
Other States population numbers not found
Other Countries population numbers not found


/home/swiadmin/Incovid19/incovid19/env/lib/python3.6/site-packages/ipykernel_launcher.py:258: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/swiadmin/Incovid19/incovid19/env/lib/python3.6/site-packages/ipykernel_launcher.py:260: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Running 7DMA for AP


Arunachal Pradesh
---------------------------------
Kra Daadi population numbers not found
Lepa Rada population numbers not found
Pakke Kessang population numbers not found
Running 7DMA for AR


Assam
---------------------------------
Airport Quarantine population numbers not found
Charaideo population numbers not found
Other State population numbers not found
Unknown population numbers not found
Running 7DMA for AS


Bihar
---------------------------------


/home/swiadmin/Incovid19/incovid19/srcDerivedValues/get7DMA.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['7DmaTestedForDistrict'][df['District'] != 'Unknown'] = test_df['7DmaTestedForDistrict'][test_df['District'] != 'Unknown']


Running 7DMA for BR


Chhattisgarh
---------------------------------
Gaurela Pendra Marwahi population numbers not found
Other State population numbers not found
Running 7DMA for CT


Goa
---------------------------------
Other State population numbers not found
Unknown population numbers not found
Running 7DMA for GA


Gujarat
---------------------------------
Running 7DMA for GJ


Haryana
---------------------------------
Running 7DMA for HR


Himachal Pradesh
---------------------------------
Running 7DMA for HP


Jharkhand
---------------------------------
Unknown population numbers not found
Running 7DMA for JH


Karnataka
---------------------------------
Other State population numbers not found
Running 7DMA for KA


Kerala
---------------------------------
Other State population numbers not found
Unknown population numbers not found
Running 7DMA for KL


Madhya Pradesh
---------------------------------
Agar Malwa population numbers not found
Niwari population numbers not found
O

# Running CSV files

In [42]:
!python ../CSV_APIs_Code/main_csv.py

../CSV_APIs_Code/main_csv.py:213: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_district["Others"]=""
../CSV_APIs_Code/main_csv.py:214: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_district["State/UTCode"]=df_district["State/UTCode"].apply(lambda val: STATE_NAMES.get(val))
../CSV_APIs_Code/main_csv.py:225: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pa

In [23]:
# dates = ["24","25","26","27","28","29","30","31"]
# for _date in dates:
#     print(_date)
# updateDerivedValues("GJ","2021-10-"+_date)
# updateDerivedValues("OR","2021-10-"+_date)
# updateDerivedValues("TN","2021-12-05")
# updateDerivedValues("TN","2021-12-06")



Tamil Nadu
---------------------------------


/home/swiadmin/Incovid19/incovid19/env/lib/python3.6/site-packages/ipykernel_launcher.py:258: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/swiadmin/Incovid19/incovid19/env/lib/python3.6/site-packages/ipykernel_launcher.py:260: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Running 7DMA for TN


Tamil Nadu
---------------------------------
Running 7DMA for TN


In [206]:
def UpdateDerivedValues(StateCode:str,Date:str,):
    """
    This function 
    1. Calulates the derived columns – Delta, Delta7 (Statewise and Districtwise) ;
    2. Updates district and population data for the state and districts of the state
    3. Updates Cumulative vaccinated numbers for the state
    4. Calculates the derived numbers for the state
    
    Input: RAWCSV file of each state    
    Output: CSV file updated with the above mentioned values
    Parameters:
    StateCode: StateCode of the corresponding State
    Date: Current Date in format "YYYY-MM-DD" type-casted as string
    
    """
    #Reading CSVs
    cowin_codes=pd.read_csv("CSV/CowinAppStateAndDistrictCode.csv")
#     sources=pd.read_csv("Sources.csv")
#     state_districts=pd.read_csv("StateDistricts.csv")
    state_raw_csv=pd.read_csv(f"RAWCSV/{Date}/{StateCode}_raw.csv",index_col=False)
    state_population=pd.read_csv("CSV/StatePopulation.csv")
    district_population=pd.read_csv("CSV/DistrictPopulation.csv")
    
    
    #Check and insert cummulativeDeceasedState 
    if state_raw_csv["cumulativeDeceasedNumberForState"].sum() == 0:
        if state_raw_csv["cumulativeDeceasedNumberForDistrict"].sum() != 0:
            state_raw_csv["cumulativeDeceasedNumberForState"] = state_raw_csv["cumulativeDeceasedNumberForDistrict"].sum()
    
    #Check for columns in state_raw_csv
    
    state_raw_csv=col_check_state_raw_csv(state_raw_csv)
#     print(state_raw_csv.head())
    
    #Reading the cumulative vaccinated numbers for the state using API
    
    state_name=STATE_NAMES[StateCode]
    print("\n")
    print(state_name)
    print("---------------------------------")
    
     #Add state/ut code
    state_raw_csv["State/UTCode"]=StateCode
    
    
    vaccination_numbers=vaccination_numbers_api(state_name,Date)
    
    state_cumulative_vaccinated1=vaccination_numbers[0]
    state_cumulative_vaccinated2=vaccination_numbers[1]
    state_cumulative_vaccinated=vaccination_numbers[2]
    
    state_raw_csv.loc[state_raw_csv["State/UTCode"]==StateCode,"cumulativeVaccinated1NumberForState"]=state_cumulative_vaccinated1
    state_raw_csv.loc[state_raw_csv["State/UTCode"]==StateCode,"cumulativeVaccinated2NumberForState"]=state_cumulative_vaccinated2
    state_raw_csv.loc[state_raw_csv["State/UTCode"]==StateCode,"cumulativeVaccinatedNumberForState"]=state_cumulative_vaccinated
    
    #Reading State population numbers from static source
    state_population_number=state_population.loc[state_population["State"]==StateCode,"StatePop"]
    state_population_number.reset_index(inplace=True,drop=True)
    state_population_number=state_population_number[0]    
    state_raw_csv.loc[state_raw_csv["State/UTCode"]==StateCode,"statePopulation"]=state_population_number
    
    #Reading the cumulative vaccinated numbers for the districts of the state
    districts=[district for district in state_raw_csv["District"]]
    # print(districts)
    for district in districts:
        print(district,state_name)
        cowin_district_code=cowin_codes.loc[((cowin_codes["district_name"]==district)&(cowin_codes["state_name"]==state_name)),"district_id"]
        cowin_district_code.reset_index(inplace=True,drop=True)
        try:
            cowin_district_code=cowin_district_code[0]
        except IndexError:
            print(f"{district} not found in Cowin States Districts Code")
            district_vaccination_data=False
            pass
        except KeyError:
            print(f"{district} not found in Cowin States Districts Code")
            district_vaccination_data=False
            pass
        else:
            ls_districts_add=cowin_codes_map[str(cowin_district_code)]
            district_cumulative_vaccinated1=0
            district_cumulative_vaccinated2=0
            district_cumulative_vaccinated=0
            for dist in ls_districts_add:
                district_cumulative_vaccinated1+=vaccination_numbers[3][dist][0]
                district_cumulative_vaccinated2+=vaccination_numbers[3][dist][1]
                district_cumulative_vaccinated+=vaccination_numbers[3][dist][2]
                district_vaccination_data=True
            
            #Reading District population numbers from static source
            district_population_number=district_population.loc[((district_population["State"]==StateCode) & (district_population["District"]==district)) ,"DistrictPop"]#[0]
            district_population_number.reset_index(inplace=True,drop=True)
            try:
                district_population_number=district_population_number[0]
            except IndexError:
                print(f"{district} population numbers not found")
                pass
            except KeyError:
                print(f"{district} population numbers not found")
                pass
            else:
                state_raw_csv.loc[((state_raw_csv["State/UTCode"]==StateCode)&(state_raw_csv["District"]==district)),"districtPopulation"]=district_population_number
            
            state_raw_csv.loc[((state_raw_csv["State/UTCode"]==StateCode)&(state_raw_csv["District"]==district)),"cumulativeVaccinated1NumberForDistrict"]=district_cumulative_vaccinated1
            state_raw_csv.loc[((state_raw_csv["State/UTCode"]==StateCode)&(state_raw_csv["District"]==district)),"cumulativeVaccinated2NumberForDistrict"]=district_cumulative_vaccinated2
            state_raw_csv.loc[((state_raw_csv["State/UTCode"]==StateCode)&(state_raw_csv["District"]==district)),"cumulativeVaccinatedNumberForDistrict"]=district_cumulative_vaccinated

        #Deriving the delta numbers
        
        prev_date=datetime.strptime(Date,"%Y-%m-%d")+timedelta(days=(-1))
        prev_date_str=str(prev_date.year)+"-"+str(prev_date.month)+"-"+str(prev_date.day)
        previous_state_raw_csv=pd.read_csv(f"RAWCSV/{prev_date_str}/{StateCode}_raw.csv")
        
        
        # print(type(state_raw_csv["District"]),state_raw_csv.columns)
        
        # for district in state_raw_csv["District"]:
        # print(district)
        # print(state_raw_csv.loc[state_raw_csv["District"] == district,"cumulativeConfirmedNumberForDistrict"].item())
        # print(8*"*")
        # print(previous_state_raw_csv.loc[previous_state_raw_csv["District"] == district,"cumulativeConfirmedNumberForDistrict"])
        # print(8*"*")
        # print(state_raw_csv.loc[state_raw_csv["District"] == district,"cumulativeConfirmedNumberForDistrict"].item()-previous_state_raw_csv.loc[previous_state_raw_csv["District"] == district,"cumulativeConfirmedNumberForDistrict"].item())
        if state_raw_csv["cumulativeConfirmedNumberForDistrict"].sum() != 0:
            state_raw_csv.loc[state_raw_csv["District"] == district,"deltaConfirmedForDistrict"]=state_raw_csv.loc[state_raw_csv["District"] == district,"cumulativeConfirmedNumberForDistrict"].item()-previous_state_raw_csv.loc[previous_state_raw_csv["District"] == district,"cumulativeConfirmedNumberForDistrict"].item()

        if state_raw_csv["cumulativeDeceasedNumberForDistrict"].sum() != 0:
            state_raw_csv.loc[state_raw_csv["District"] == district,"deltaDeceasedForDistrict"]=state_raw_csv.loc[state_raw_csv["District"] == district,"cumulativeDeceasedNumberForDistrict"].item()-previous_state_raw_csv.loc[previous_state_raw_csv["District"] == district,"cumulativeDeceasedNumberForDistrict"].item()

        if state_raw_csv["cumulativeRecoveredNumberForDistrict"].sum() != 0:
            state_raw_csv.loc[state_raw_csv["District"] == district,"deltaRecoveredForDistrict"]=state_raw_csv.loc[state_raw_csv["District"] == district,"cumulativeRecoveredNumberForDistrict"].item()-previous_state_raw_csv.loc[previous_state_raw_csv["District"]  == district,"cumulativeRecoveredNumberForDistrict"].item()

        if state_raw_csv["cumulativeTestedNumberForDistrict"].sum() != 0:
            state_raw_csv.loc[state_raw_csv["District"] == district,"deltaTestedForDistrict"]=state_raw_csv.loc[state_raw_csv["District"] == district,"cumulativeTestedNumberForDistrict"].item()-previous_state_raw_csv.loc[previous_state_raw_csv["District"] == district,"cumulativeTestedNumberForDistrict"].item()

        if state_raw_csv["cumulativeVaccinatedNumberForDistrict"].sum() != 0:
            state_raw_csv.loc[state_raw_csv["District"] == district,"deltaVaccinatedForDistrict"]=state_raw_csv.loc[state_raw_csv["District"] == district,"cumulativeVaccinatedNumberForDistrict"].item()-previous_state_raw_csv.loc[previous_state_raw_csv["District"] == district,"cumulativeVaccinatedNumberForDistrict"].item()

        if state_raw_csv["cumulativeVaccinated1NumberForDistrict"].sum() != 0:
            state_raw_csv.loc[state_raw_csv["District"] == district,"deltaVaccinated1ForDistrict"]=state_raw_csv.loc[state_raw_csv["District"] == district,"cumulativeVaccinated1NumberForDistrict"].item()-previous_state_raw_csv.loc[previous_state_raw_csv["District"] == district,"cumulativeVaccinated1NumberForDistrict"].item()

        if state_raw_csv["cumulativeVaccinated2NumberForDistrict"].sum() != 0:
            state_raw_csv.loc[state_raw_csv["District"] == district,"deltaVaccinated2ForDistrict"]=state_raw_csv.loc[state_raw_csv["District"] == district,"cumulativeVaccinated2NumberForDistrict"].item()-previous_state_raw_csv.loc[previous_state_raw_csv["District"] == district,"cumulativeVaccinated2NumberForDistrict"].item()
            
                
        if state_raw_csv["cumulativeConfirmedNumberForState"].sum() != 0:
            # print(state_raw_csv["cumulativeConfirmedNumberForState"],type(state_raw_csv["cumulativeConfirmedNumberForState"]))
            state_raw_csv["deltaConfirmedForState"]=state_raw_csv["cumulativeConfirmedNumberForState"]-previous_state_raw_csv["cumulativeConfirmedNumberForState"]
        
        if state_raw_csv["cumulativeDeceasedNumberForState"].sum() != 0:
            state_raw_csv["deltaDeceasedForState"]=state_raw_csv["cumulativeDeceasedNumberForState"]-previous_state_raw_csv["cumulativeDeceasedNumberForState"]
        
        if state_raw_csv["cumulativeRecoveredNumberForState"].sum() != 0:
            state_raw_csv["deltaRecoveredForState"]=state_raw_csv["cumulativeRecoveredNumberForState"]-previous_state_raw_csv["cumulativeRecoveredNumberForState"]
        
        if state_raw_csv["cumulativeVaccinatedNumberForState"].sum() != 0:
            # print("today")
            # print(state_raw_csv["cumulativeVaccinatedNumberForState"])
            # print("yesterday")
            # print(previous_state_raw_csv["cumulativeVaccinatedNumberForState"])
            # print("Difference")
            # print(state_raw_csv["cumulativeVaccinatedNumberForState"]-previous_state_raw_csv["cumulativeVaccinatedNumberForState"])
            state_raw_csv["deltaVaccinatedForState"]=state_raw_csv["cumulativeVaccinatedNumberForState"][0]-previous_state_raw_csv["cumulativeVaccinatedNumberForState"][0]
        
        state_raw_csv["deltaVaccinated1ForState"]=state_raw_csv["cumulativeVaccinated1NumberForState"]-previous_state_raw_csv["cumulativeVaccinated1NumberForState"]
        state_raw_csv["deltaVaccinated2ForState"]=state_raw_csv["cumulativeVaccinated2NumberForState"]-previous_state_raw_csv["cumulativeVaccinated2NumberForState"]
        
        
        if state_raw_csv["cumulativeTestedNumberForState"].sum() != 0:
            state_raw_csv["deltaTestedForState"]=state_raw_csv["cumulativeTestedNumberForState"]-previous_state_raw_csv["cumulativeTestedNumberForState"]
        
       
        
        if district_vaccination_data:
            state_raw_csv["deltaVaccinatedForDistrict"]=state_raw_csv["cumulativeVaccinatedNumberForDistrict"]-previous_state_raw_csv["cumulativeVaccinatedNumberForDistrict"]
        avg_counter=1
        
        state_raw_csv["delta7ConfirmedForState"]=0
        state_raw_csv["delta7DeceasedForState"]=0
        state_raw_csv["delta7RecoveredForState"]=0
        state_raw_csv["delta7VaccinatedForState"]=0
        state_raw_csv["delta7TestedForState"]=0
        
        state_raw_csv["delta7ConfirmedForDistrict"]=0
        state_raw_csv["delta7DeceasedForDistrict"]=0
        state_raw_csv["delta7RecoveredForDistrict"]=0
        state_raw_csv["delta7VaccinatedForDistrict"]=0
        state_raw_csv["delta7TestedForState"]=0
        
        #Logic
        #delta7:# on (today) - # of (today-7)
        #delta21_14_: # on (today-21) - # of (today-14)
        #7dma: {(# of today)+(# of today-1)+(# of today-2)+(# of today-3)+(# of today-4)+(# of today-5+# of today-6)}/7
        
        prev_date7=datetime.strptime(Date,"%Y-%m-%d")+timedelta(days=-7)
        prev_date_str=str(prev_date7.year)+"-"+str(prev_date7.month)+"-"+str(prev_date7.day)
        previous_state_raw_csv=pd.read_csv(f"RAWCSV/{prev_date_str}/{StateCode}_raw.csv")
        
        # print(state_raw_csv["cumulativeConfirmedNumberForState"],type(state_raw_csv["cumulativeConfirmedNumberForState"]))
        state_raw_csv["delta7ConfirmedForState"]=state_raw_csv["cumulativeConfirmedNumberForState"]-previous_state_raw_csv["cumulativeConfirmedNumberForState"]
        state_raw_csv["delta7DeceasedForState"]=state_raw_csv["cumulativeDeceasedNumberForState"]-previous_state_raw_csv["cumulativeDeceasedNumberForState"]
        state_raw_csv["delta7RecoveredForState"]=state_raw_csv["cumulativeRecoveredNumberForState"]-previous_state_raw_csv["cumulativeRecoveredNumberForState"]
        state_raw_csv["delta7VaccinatedForState"]=state_raw_csv["cumulativeVaccinatedNumberForState"]-previous_state_raw_csv["cumulativeVaccinatedNumberForState"]
        state_raw_csv["delta7TestedForState"]=state_raw_csv["cumulativeTestedNumberForState"]-previous_state_raw_csv["cumulativeTestedNumberForState"]

        state_raw_csv["delta7ConfirmedForDistrict"]=state_raw_csv["cumulativeConfirmedNumberForDistrict"]-previous_state_raw_csv["cumulativeConfirmedNumberForDistrict"]
        state_raw_csv["delta7DeceasedForDistrict"]=state_raw_csv["cumulativeDeceasedNumberForDistrict"]-previous_state_raw_csv["cumulativeDeceasedNumberForDistrict"]
        state_raw_csv["delta7RecoveredForDistrict"]=state_raw_csv["cumulativeRecoveredNumberForDistrict"]-previous_state_raw_csv["cumulativeRecoveredNumberForDistrict"]
        state_raw_csv["delta7VaccinatedForDistrict"]=state_raw_csv["cumulativeVaccinatedNumberForDistrict"]-previous_state_raw_csv["cumulativeVaccinatedNumberForDistrict"]
        state_raw_csv["delta7VaccinatedForDistrict"]=state_raw_csv["cumulativeTestedNumberForDistrict"]-previous_state_raw_csv["cumulativeTestedNumberForDistrict"]

        
        prev_date21=datetime.strptime(Date,"%Y-%m-%d")+timedelta(days=-21)
        prev_date_21_str=prev_date21.strftime("%Y-%m-%d")
#         str(prev_date21.year)+"-"+str(prev_date21.month)+"-"+str(prev_date21.day)
        previous_state_21_raw_csv=pd.read_csv(f"RAWCSV/{prev_date_21_str}/{StateCode}_raw.csv")
        
        prev_date14=datetime.strptime(Date,"%Y-%m-%d")+timedelta(days=-14)
        prev_date_14_str=str(prev_date14.year)+"-"+str(prev_date14.month)+"-"+str(prev_date14.day)
        previous_state_14_raw_csv=pd.read_csv(f"RAWCSV/{prev_date_14_str}/{StateCode}_raw.csv")
        
        state_raw_csv["delta21_14ConfirmedForState"]=previous_state_14_raw_csv["cumulativeConfirmedNumberForState"]-previous_state_21_raw_csv["cumulativeConfirmedNumberForState"]
        
        # for district in state_raw_csv["District"]:
        state_raw_csv.loc[state_raw_csv["District"] == district,"delta21_14ConfirmedForDistrict"]=previous_state_14_raw_csv.loc[previous_state_14_raw_csv["District"] == district,"cumulativeConfirmedNumberForDistrict"].item()-previous_state_21_raw_csv.loc[previous_state_21_raw_csv["District"] == district,"cumulativeConfirmedNumberForDistrict"].item()
                
        state_raw_csv.loc[:,"7DmaConfirmedForState"]=0
        state_raw_csv.loc[:,"7DmaDeceasedForState"]=0
        state_raw_csv.loc[:,"7DmaRecoveredForState"]=0
        state_raw_csv.loc[:,"7DmaVaccinatedForState"]=0
        state_raw_csv.loc[:,"7DmaTestedForState"]=0
        state_raw_csv.loc[:,"7DmaVaccinated1ForState"]=0
        state_raw_csv.loc[:,"7DmaVaccinated2ForState"]=0

        state_raw_csv.loc[:,"7DmaConfirmedForDistrict"]=0
        state_raw_csv.loc[:,"7DmaDeceasedForDistrict"]=0
        state_raw_csv.loc[:,"7DmaRecoveredForDistrict"]=0
        state_raw_csv.loc[:,"7DmaVaccinatedForDistrict"]=0
        state_raw_csv.loc[:,"7DmaTestedForDistrict"]=0
        state_raw_csv.loc[:,"7DmaVaccinated1ForDistrict"]=0
        state_raw_csv.loc[:,"7DmaVaccinated2ForDistrict"]=0
        x=0
        
        
        avg_counter = 0
        for i in range(7):
            prev_date=datetime.strptime(Date,"%Y-%m-%d")+timedelta(days=(-1*i))
            print(str(prev_date))
            prev_date_str=str(prev_date.year)+"-"+str(prev_date.month)+"-"+str(prev_date.day)
            previous_state_raw_csv=pd.read_csv(f"RAWCSV/{prev_date_str}/{StateCode}_raw.csv")
            
            state_raw_csv["7DmaConfirmedForState"]+=previous_state_raw_csv["cumulativeConfirmedNumberForState"]
            state_raw_csv["7DmaDeceasedForState"]+=previous_state_raw_csv["cumulativeDeceasedNumberForState"]
            state_raw_csv["7DmaRecoveredForState"]+=previous_state_raw_csv["cumulativeRecoveredNumberForState"]
            state_raw_csv["7DmaVaccinatedForState"]+=previous_state_raw_csv["cumulativeVaccinatedNumberForState"]
            state_raw_csv["7DmaTestedForState"]+=previous_state_raw_csv["cumulativeTestedNumberForState"]
            state_raw_csv["7DmaVaccinated1ForState"]+=previous_state_raw_csv["cumulativeVaccinated1NumberForState"]
            state_raw_csv["7DmaVaccinated2ForState"]+=previous_state_raw_csv["cumulativeVaccinated2NumberForState"]
            
            
#             x=x+state_raw_csv.loc[state_raw_csv["District"] == district,"7DmaVaccinatedForDistrict"].item() + previous_state_raw_csv.loc[previous_state_raw_csv["District"] == district,"cumulativeVaccinatedNumberForDistrict"].item()
#             y=previous_state_raw_csv.loc[previous_state_raw_csv["District"] == district,"cumulativeVaccinatedNumberForDistrict"].item()
#             print(x,y)
#             print(x+y)
            avg_counter+=1
        # state_raw_csv.loc[state_raw_csv["District"] == district,"7DmaVaccinatedForDistrict"] = x
        # print(type(state_raw_csv.loc[state_raw_csv.loc[state_raw_csv["District"] == district].index.item(),"7DmaVaccinatedForDistrict"]))
        # state_raw_csv.loc[state_raw_csv.loc[state_raw_csv["District"] == district].index.item(),"7DmaVaccinatedForDistrict"] = float(x)
        # test.loc[test.loc[test["max_speed"] == 4,"shield"].index.item(),"shield"]
        # print(state_raw_csv["7DmaVaccinatedForDistrict"])# = x
#             # for district in state_raw_csv["District"]:
#             # state_raw_csv["7DmaVaccinatedForDistrict"] = state_raw_csv["7DmaVaccinatedForDistrict"].astype('float')
#             state_raw_csv.loc[state_raw_csv["District"] == district,"7DmaConfirmedForDistrict"] = state_raw_csv.loc[state_raw_csv["District"] == district,"7DmaConfirmedForDistrict"].item() + previous_state_raw_csv.loc[previous_state_raw_csv["District"] == district,"cumulativeConfirmedNumberForDistrict"].item()
#             state_raw_csv.loc[state_raw_csv["District"] == district,"7DmaDeceasedForDistrict"] = state_raw_csv.loc[state_raw_csv["District"] == district,"7DmaDeceasedForDistrict"].item() + previous_state_raw_csv.loc[previous_state_raw_csv["District"] == district,"cumulativeDeceasedNumberForDistrict"].item()
#             state_raw_csv.loc[state_raw_csv["District"] == district,"7DmaRecoveredForDistrict"] = state_raw_csv.loc[state_raw_csv["District"] == district,"7DmaRecoveredForDistrict"].item() + previous_state_raw_csv.loc[previous_state_raw_csv["District"] == district,"cumulativeRecoveredNumberForDistrict"].item()
#             state_raw_csv.loc[state_raw_csv["District"] == district,"7DmaVaccinatedForDistrict"][state_raw_csv.loc[state_raw_csv["District"] == district,"7DmaVaccinatedForDistrict"].index] = state_raw_csv.loc[state_raw_csv["District"] == district,"7DmaVaccinatedForDistrict"].item() + previous_state_raw_csv.loc[previous_state_raw_csv["District"] == district,"cumulativeVaccinatedNumberForDistrict"].item()
#             print("District Vac")
#             print(state_raw_csv.loc[state_raw_csv["District"] == district,"7DmaConfirmedForDistrict"].index)
#             print(previous_state_raw_csv.loc[previous_state_raw_csv["District"] == district,"cumulativeVaccinatedNumberForDistrict"])
#             print("Vac Total")
#             print(state_raw_csv.loc[state_raw_csv["District"] == district,"7DmaVaccinatedForDistrict"])
#             state_raw_csv.loc[state_raw_csv["District"] == district,"7DmaTestedForDistrict"] = state_raw_csv.loc[state_raw_csv["District"] == district,"7DmaTestedForDistrict"].item() + previous_state_raw_csv.loc[previous_state_raw_csv["District"] == district,"cumulativeTestedNumberForDistrict"].item()
#             state_raw_csv.loc[state_raw_csv["District"] == district,"7DmaVaccinated1ForDistrict"] = state_raw_csv.loc[state_raw_csv["District"] == district,"7DmaVaccinated1ForDistrict"].item()+previous_state_raw_csv.loc[previous_state_raw_csv["District"] == district,"cumulativeVaccinated1NumberForDistrict"].item()
#             state_raw_csv.loc[state_raw_csv["District"] == district,"7DmaVaccinated2ForDistrict"] = state_raw_csv.loc[state_raw_csv["District"] == district,"7DmaVaccinated2ForDistrict"].item() + previous_state_raw_csv.loc[previous_state_raw_csv["District"] == district,"cumulativeVaccinated2NumberForDistrict"].item()

            
            
        state_raw_csv["7DmaConfirmedForState"]=round(state_raw_csv["7DmaConfirmedForState"]/avg_counter,0)
        state_raw_csv["7DmaDeceasedForState"]=round(state_raw_csv["7DmaDeceasedForState"]/avg_counter,0)
        state_raw_csv["7DmaRecoveredForState"]=round(state_raw_csv["7DmaRecoveredForState"]/avg_counter,0)
        state_raw_csv["7DmaVaccinatedForState"]=round(state_raw_csv["7DmaVaccinatedForState"]/avg_counter,0)
        state_raw_csv["7DmaTestedForState"]=round(state_raw_csv["7DmaTestedForState"]/avg_counter,0)
        state_raw_csv["7DmaVaccinated1ForState"]=round(state_raw_csv["7DmaVaccinated1ForState"]/avg_counter,0)
        state_raw_csv["7DmaVaccinated2ForState"]=round(state_raw_csv["7DmaVaccinated2ForState"]/avg_counter,0)
        
        
        
#         state_raw_csv.loc[state_raw_csv["District"] == district,"7DmaConfirmedForDistrict"]=round(state_raw_csv.loc[state_raw_csv["District"] == district,"7DmaConfirmedForDistrict"].item()/avg_counter,0)
#         state_raw_csv.loc[state_raw_csv["District"] == district,"7DmaDeceasedForDistrict"]=round(state_raw_csv.loc[state_raw_csv["District"] == district,"7DmaDeceasedForDistrict"].item()/avg_counter,0)
#         state_raw_csv.loc[state_raw_csv["District"] == district,"7DmaRecoveredForDistrict"]=round(state_raw_csv.loc[state_raw_csv["District"] == district,"7DmaRecoveredForDistrict"].item()/avg_counter,0)
#         # if state_raw_csv.loc[state_raw_csv["District"] == district,"7DmaVaccinatedForDistrict"].isna().item():
#         #     state_raw_csv.loc[state_raw_csv["District"] == district,"7DmaVaccinatedForDistrict"] = 0
#         # else:
#         # state_raw_csv.loc[state_raw_csv["District"] == district,"7DmaVaccinatedForDistrict"] = round(state_raw_csv.loc[state_raw_csv["District"] == district,"7DmaVaccinatedForDistrict"].item()/avg_counter,0)
#         state_raw_csv.loc[state_raw_csv["District"] == district,"7DmaTestedForDistrict"]=round(state_raw_csv.loc[state_raw_csv["District"] == district,"7DmaTestedForDistrict"].item()/avg_counter,0)
#         state_raw_csv.loc[state_raw_csv["District"] == district,"7DmaVaccinated1ForDistrict"]=round(state_raw_csv.loc[state_raw_csv["District"] == district,"7DmaVaccinated1ForDistrict"].item()/avg_counter,0)
#         state_raw_csv.loc[state_raw_csv["District"] == district,"7DmaVaccinated2ForDistrict"]=round(state_raw_csv.loc[state_raw_csv["District"] == district,"7DmaVaccinated2ForDistrict"].item()/avg_counter,0)
        
        # state_raw_csv["7DmaVaccinatedForDistrict"] = round(state_raw_csv["7DmaVaccinatedForDistrict"]/avg_counter,0)
        
        
#         print("Vac Avg")
#         print(round(state_raw_csv.loc[state_raw_csv["District"] == district,"7DmaVaccinatedForDistrict"]/avg_counter,0))
#         print(10*"&")
#         print(state_raw_csv["7DmaVaccinatedForDistrict"])



        # if state_raw_csv.loc[state_raw_csv["District"] == district,"7DmaVaccinatedForDistrict"].isna().item():
        #     print("True")
            # print(int(round(state_raw_csv.loc[state_raw_csv["District"] == district,"7DmaVaccinatedForDistrict"].item()/avg_counter,0)))
        
#         state_raw_csv["delta21_14ConfirmedForState"]=previous_state_raw_csv["cumulativeConfirmedNumberForState"]
        
#         state_raw_csv["delta14ConfirmedForState"]=0
#         state_raw_csv["delta21ConfirmedForState"]=0
        
#         for i in range(8,15):
#             prev_date=datetime.strptime(Date,"%Y-%m-%d")+timedelta(days=(-1*i))
#             prev_date_str=str(prev_date.year)+"-"+str(prev_date.month)+"-"+str(prev_date.day)
#             previous_state_raw_csv=pd.read_csv(f"./RAWCSV/{prev_date_str}/{StateCode}_raw.csv")
           
#             state_raw_csv["delta14ConfirmedForState"]+=previous_state_raw_csv["cumulativeConfirmedNumberForState"]
            
#         state_raw_csv["delta21ConfirmedForState"]=state_raw_csv["delta14ConfirmedForState"]
#         for i in range(15,22):
#             prev_date=datetime.strptime(Date,"%Y-%m-%d")+timedelta(days=(-1*i))
#             prev_date_str=prev_date.strftime('%Y-%m-%d')
# #             print(prev_date,prev_date_str)
#             previous_state_raw_csv=pd.read_csv(f"./RAWCSV/{prev_date_str}/{StateCode}_raw.csv")
#             state_raw_csv["delta21ConfirmedForState"]+=previous_state_raw_csv["cumulativeConfirmedNumberForState"]
#         state_raw_csv["delta21_14ConfirmedForState"]=state_raw_csv["delta21ConfirmedForState"]-state_raw_csv["delta14ConfirmedForState"]                
    
    lst = ["Recovered","Confirmed","Deceased","Tested","Vaccinated1","Vaccinated2"]
    for val in lst:
        for idx in state_raw_csv.index:
            # print(new_state_raw_csv["District"][idx])
            # print(new_state_raw_csv["7DmaVaccinatedForDistrict"][idx])
            avg_counter = 0
            value = 0
            for i in range(1,8):
                prev_date=datetime.strptime(Date,"%Y-%m-%d")+timedelta(days=(-1*i))
                prev_date_str=str(prev_date.year)+"-"+str(prev_date.month)+"-"+str(prev_date.day)
                previous_state_raw_csv=pd.read_csv(f"RAWCSV/{prev_date_str}/{StateCode}_raw.csv")
                value += previous_state_raw_csv.loc[previous_state_raw_csv["District"] == state_raw_csv["District"][idx],"cumulative{}NumberForDistrict".format(val)]
                avg_counter += 1
            state_raw_csv["7Dma{}ForDistrict".format(val)][idx] = value.item()/avg_counter

        state_raw_csv.to_csv(f"RAWCSV/{Date}/{StateCode}_final.csv",index=False)
        return(state_raw_csv)
            

In [207]:
# keys=["AR","BR","CT","HP","MN","RJ"]
keys = ["RJ"]
for key in keys:
    new_state_raw_csv=UpdateDerivedValues(key,"2021-10-27")



Rajasthan
---------------------------------
29
Ajmer Rajasthan
2021-10-26 00:00:00
2021-10-25 00:00:00
2021-10-24 00:00:00
2021-10-23 00:00:00
2021-10-22 00:00:00
2021-10-21 00:00:00
2021-10-20 00:00:00
Alwar Rajasthan
2021-10-26 00:00:00
2021-10-25 00:00:00
2021-10-24 00:00:00
2021-10-23 00:00:00
2021-10-22 00:00:00
2021-10-21 00:00:00
2021-10-20 00:00:00
Banswara Rajasthan
2021-10-26 00:00:00
2021-10-25 00:00:00
2021-10-24 00:00:00
2021-10-23 00:00:00
2021-10-22 00:00:00
2021-10-21 00:00:00
2021-10-20 00:00:00
Baran Rajasthan
2021-10-26 00:00:00
2021-10-25 00:00:00
2021-10-24 00:00:00
2021-10-23 00:00:00
2021-10-22 00:00:00
2021-10-21 00:00:00
2021-10-20 00:00:00
Barmer Rajasthan
2021-10-26 00:00:00
2021-10-25 00:00:00
2021-10-24 00:00:00
2021-10-23 00:00:00
2021-10-22 00:00:00
2021-10-21 00:00:00
2021-10-20 00:00:00
Bharatpur Rajasthan
2021-10-26 00:00:00
2021-10-25 00:00:00
2021-10-24 00:00:00
2021-10-23 00:00:00
2021-10-22 00:00:00
2021-10-21 00:00:00
2021-10-20 00:00:00
Bhilwar

c:\users\vishnuprasadp\incovid19\incovid19\env\lib\site-packages\ipykernel_launcher.py:357: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [202]:
Date = "2021-10-27"
StateCode = "RJ"
new_state_raw_csv
lst = ["Recovered","Confirmed","Deceased","Tested","Vaccinated1","Vaccinated2"]
for val in lst:
    for idx in new_state_raw_csv.index:
        # print(new_state_raw_csv["District"][idx])
        # print(new_state_raw_csv["7DmaVaccinatedForDistrict"][idx])
        avg_counter = 0
        value = 0
        for i in range(1,8):
            prev_date=datetime.strptime(Date,"%Y-%m-%d")+timedelta(days=(-1*i))
            prev_date_str=str(prev_date.year)+"-"+str(prev_date.month)+"-"+str(prev_date.day)
            previous_state_raw_csv=pd.read_csv(f"RAWCSV/{prev_date_str}/{StateCode}_raw.csv")
            value += previous_state_raw_csv.loc[previous_state_raw_csv["District"] == new_state_raw_csv["District"][idx],"cumulative{}NumberForDistrict".format(val)]
            avg_counter += 1
        new_state_raw_csv["7Dma{}ForDistrict".format(val)][idx] = value.item()/avg_counter

c:\users\vishnuprasadp\incovid19\incovid19\env\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
c:\users\vishnuprasadp\incovid19\incovid19\env\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [210]:
new_state_raw_csv["7DmaVaccinated1ForDistrict"]

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
15    0
16    0
17    0
18    0
19    0
20    0
21    0
22    0
23    0
24    0
25    0
26    0
27    0
28    0
29    0
30    0
31    0
32    0
33    0
Name: 7DmaVaccinated1ForDistrict, dtype: int64

In [12]:
import pandas as pd
state_raw_csv=pd.read_csv(f"RAWCSV/2021-10-27/RJ_raw.csv",index_col=False)

In [13]:
state_raw_csv.columns

Index(['Date', 'State/UTCode', 'District', 'tested_last_updated_district',
       'tested_source_district', 'notesForDistrict',
       'cumulativeConfirmedNumberForDistrict',
       'cumulativeDeceasedNumberForDistrict',
       'cumulativeRecoveredNumberForDistrict',
       'cumulativeTestedNumberForDistrict', 'last_updated',
       'tested_last_updated_state', 'tested_source_state', 'notesForState',
       'cumulativeConfirmedNumberForState', 'cumulativeDeceasedNumberForState',
       'cumulativeRecoveredNumberForState', 'cumulativeTestedNumberForState'],
      dtype='object')

In [14]:
if state_raw_csv["cumulativeDeceasedNumberForState"].sum() == 0:
    if state_raw_csv["cumulativeDeceasedNumberForDistrict"].sum() != 0:
        state_raw_csv["cumulativeDeceasedNumberForState"] = state_raw_csv["cumulativeDeceasedNumberForDistrict"].sum()

In [16]:
state_raw_csv

,Date,State/UTCode,District,tested_last_updated_district,tested_source_district,notesForDistrict,cumulativeConfirmedNumberForDistrict,cumulativeDeceasedNumberForDistrict,cumulativeRecoveredNumberForDistrict,cumulativeTestedNumberForDistrict,last_updated,tested_last_updated_state,tested_source_state,notesForState,cumulativeConfirmedNumberForState,cumulativeDeceasedNumberForState,cumulativeRecoveredNumberForState,cumulativeTestedNumberForState
0,10/27/2021,Rajasthan,Ajmer,2021-10-27 18:00:00+05:30,https://twitter.com/dineshkumawat,NaN,37766,410,37351,602701,2021-10-27 18:00:00+05:30,2021-10-27 18:00:00+05:30,https://twitter.com/dineshkumawat,NaN,954411,8954,945437,14752007
1,10/27/2021,Rajasthan,Alwar,2021-10-27 18:00:00+05:30,https://twitter.com/dineshkumawat,NaN,59694,307,59387,1199452,2021-10-27 18:00:00+05:30,2021-10-27 18:00:00+05:30,https://twitter.com/dineshkumawat,NaN,954411,8954,945437,14752007
2,10/27/2021,Rajasthan,Banswara,2021-10-27 18:00:00+05:30,https://twitter.com/dineshkumawat,NaN,10005,104,9901,215225,2021-10-27 18:00:00+05:30,2021-10-27 18:00:00+05:30,https://twitter.com/dineshkumawat,NaN,954411,8954,945437,14752007
3,10/27/2021,Rajasthan,Baran,2021-10-27 18:00:00+05:30,https://twitter.com/dineshkumawat,NaN,11995,61,11934,148944,2021-10-27 18:00:00+05:30,2021-10-27 18:00:00+05:30,https://twitter.com/dineshkumawat,NaN,954411,8954,945437,14752007
4,10/27/2021,Rajasthan,Barmer,2021-10-27 18:00:00+05:30,https://twitter.com/dineshkumawat,NaN,15562,185,15377,321338,2021-10-27 18:00:00+05:30,2021-10-27 18:00:00+05:30,https://twitter.com/dineshkumawat,NaN,954411,8954,945437,14752007
5,10/27/2021,Rajasthan,Bharatpur,2021-10-27 18:00:00+05:30,https://twitter.com/dineshkumawat,NaN,19601,260,19341,378048,2021-10-27 18:00:00+05:30,2021-10-27 18:00:00+05:30,https://twitter.com/dineshkumawat,NaN,954411,8954,945437,14752007
6,10/27/2021,Rajasthan,Bhilwara,2021-10-27 18:00:00+05:30,https://twitter.com/dineshkumawat,NaN,29781,156,29625,375245,2021-10-27 18:00:00+05:30,2021-10-27 18:00:00+05:30,https://twitter.com/dineshkumawat,NaN,954411,8954,945437,14752007
7,10/27/2021,Rajasthan,Bikaner,2021-10-27 18:00:00+05:30,https://twitter.com/dineshkumawat,NaN,40337,545,39791,581705,2021-10-27 18:00:00+05:30,2021-10-27 18:00:00+05:30,https://twitter.com/dineshkumawat,NaN,954411,8954,945437,14752007
8,10/27/2021,Rajasthan,Bundi,2021-10-27 18:00:00+05:30,https://twitter.com/dineshkumawat,NaN,7967,48,7919,167602,2021-10-27 18:00:00+05:30,2021-10-27 18:00:00+05:30,https://twitter.com/dineshkumawat,NaN,954411,8954,945437,14752007
9,10/27/2021,Rajasthan,Chittorgarh,2021-10-27 18:00:00+05:30,https://twitter.com/dineshkumawat,NaN,19809,139,19670,348600,2021-10-27 18:00:00+05:30,2021-10-27 18:00:00+05:30,https://twitter.com/dineshkumawat,NaN,954411,8954,945437,14752007


In [ ]:
cowin_codes.loc[((cowin_codes["district_name"]==district)&(cowin_codes["state_name"]==state_name)),"district_id"]

In [25]:
cowin_codes.loc[((cowin_codes["state_name"]=="Andhra Pradesh"))]

,state_id,state_name,district_id,district_name,Discrepancy,district_name_cowin,Remark,district_id_add
3,2,Andhra Pradesh,9,Anantapur,Anantapur,Anantapur,NaN,NULL
4,2,Andhra Pradesh,10,Chittoor,Chittoor,Chittoor,NaN,NULL
5,2,Andhra Pradesh,11,East Godavari,East Godavari,East Godavari,NaN,NULL
6,2,Andhra Pradesh,5,Guntur,Guntur,Guntur,NaN,NULL
7,2,Andhra Pradesh,4,Krishna,Krishna,Krishna,NaN,NULL
8,2,Andhra Pradesh,7,Kurnool,Kurnool,Kurnool,NaN,NULL
9,2,Andhra Pradesh,12,Prakasam,Prakasam,Prakasam,NaN,NULL
10,2,Andhra Pradesh,13,S.P.S. Nellore,S.P.S. Nellore,Sri Potti Sriramulu Nellore,NaN,NULL
11,2,Andhra Pradesh,14,Srikakulam,Srikakulam,Srikakulam,NaN,NULL
12,2,Andhra Pradesh,8,Visakhapatnam,Visakhapatnam,Visakhapatnam,NaN,NULL


In [22]:
Ananthapur

,state_id,state_name,district_id,district_name,Discrepancy,district_name_cowin,Remark,district_id_add
0,1,Andaman and Nicobar Islands,3,Nicobars,Nicobars,Nicobar,NaN,NULL
1,1,Andaman and Nicobar Islands,1,North and Middle Andaman,North and Middle Andaman,North and Middle Andaman,NaN,NULL
2,1,Andaman and Nicobar Islands,2,South Andaman,South Andaman,South Andaman,NaN,NULL
3,2,Andhra Pradesh,9,Anantapur,Anantapur,Anantapur,NaN,NULL
4,2,Andhra Pradesh,10,Chittoor,Chittoor,Chittoor,NaN,NULL
...,...,...,...,...,...,...,...,...
751,36,West Bengal,733,Purulia,Purulia,Purulia,NaN,NULL
752,36,West Bengal,734,Birbhum,Birbhum,Rampurhat HD (Birbhum),NaN,NULL
753,36,West Bengal,735,South 24 Parganas,South 24 Parganas,South 24 Parganas,NaN,NULL
754,36,West Bengal,736,Uttar Dinajpur,Uttar Dinajpur,Uttar Dinajpur,NaN,NULL


In [17]:
previous_state_raw_csv=pd.read_csv(f"RAWCSV/2021-10-26/RJ_raw.csv")

In [18]:
previous_state_raw_csv

,Date,State/UTCode,deltaConfirmedForState,deltaDeceasedForState,deltaRecoveredForState,deltaTestedForState,deltaVaccinated1ForState,deltaVaccinated2ForState,delta21_14confirmedForState,7DmaConfirmedForState,...,tested_source_state,notesForState,cumulativeConfirmedNumberForState,cumulativeDeceasedNumberForState,cumulativeRecoveredNumberForState,cumulativeTestedNumberForState,cumulativeVaccinated1NumberForState,cumulativeVaccinated2NumberForState,cumulativeVaccinatedNumberForState,cumulativeVaccinatedNumberForDistrict
0,2021-10-26,RJ,0,0,0,0,0,0,0,0,...,https://twitter.com/dineshkumawat,NaN,954407,0,945433,14735295,42355630,19460720,61816350.0,2742662.0
1,2021-10-26,RJ,0,0,0,0,0,0,0,0,...,https://twitter.com/dineshkumawat,NaN,954407,0,945433,14735295,42355630,19460720,61816350.0,3237352.0
2,2021-10-26,RJ,0,0,0,0,0,0,0,0,...,https://twitter.com/dineshkumawat,NaN,954407,0,945433,14735295,42355630,19460720,61816350.0,1529401.0
3,2021-10-26,RJ,0,0,0,0,0,0,0,0,...,https://twitter.com/dineshkumawat,NaN,954407,0,945433,14735295,42355630,19460720,61816350.0,1046655.0
4,2021-10-26,RJ,0,0,0,0,0,0,0,0,...,https://twitter.com/dineshkumawat,NaN,954407,0,945433,14735295,42355630,19460720,61816350.0,2063038.0
5,2021-10-26,RJ,0,0,0,0,0,0,0,0,...,https://twitter.com/dineshkumawat,NaN,954407,0,945433,14735295,42355630,19460720,61816350.0,1916809.0
6,2021-10-26,RJ,0,0,0,0,0,0,0,0,...,https://twitter.com/dineshkumawat,NaN,954407,0,945433,14735295,42355630,19460720,61816350.0,2228607.0
7,2021-10-26,RJ,0,0,0,0,0,0,0,0,...,https://twitter.com/dineshkumawat,NaN,954407,0,945433,14735295,42355630,19460720,61816350.0,2156620.0
8,2021-10-26,RJ,0,0,0,0,0,0,0,0,...,https://twitter.com/dineshkumawat,NaN,954407,0,945433,14735295,42355630,19460720,61816350.0,1046541.0
9,2021-10-26,RJ,0,0,0,0,0,0,0,0,...,https://twitter.com/dineshkumawat,NaN,954407,0,945433,14735295,42355630,19460720,61816350.0,1434740.0


In [22]:
# state_raw_csv["cumulativeVaccinatedNumberForState"]
previous_state_raw_csv["cumulativeVaccinatedNumberForState"]

0     61816350.0
1     61816350.0
2     61816350.0
3     61816350.0
4     61816350.0
5     61816350.0
6     61816350.0
7     61816350.0
8     61816350.0
9     61816350.0
10    61816350.0
11    61816350.0
12    61816350.0
13    61816350.0
14    61816350.0
15    61816350.0
16    61816350.0
17    61816350.0
18    61816350.0
19    61816350.0
20    61816350.0
21    61816350.0
22    61816350.0
23    61816350.0
24    61816350.0
25    61816350.0
26    61816350.0
27    61816350.0
28    61816350.0
29    61816350.0
30    61816350.0
31    61816350.0
32    61816350.0
33    61816350.0
Name: cumulativeVaccinatedNumberForState, dtype: float64

In [158]:
test = pd.DataFrame([[1, 2], [4, 5], [7, 8]],
     index=['cobra', 'viper', 'sidewinder'],
     columns=['max_speed', 'shield'])

In [166]:
type(test.loc['cobra', 'shield'])

numpy.int64

In [172]:
test.loc[test["max_speed"] == 4,"shield"].index.item()

'viper'

In [173]:
test.loc[test.loc[test["max_speed"] == 4,"shield"].index.item(),"shield"]

5

In [162]:
test

,max_speed,shield
cobra,1,10
viper,4,5
sidewinder,7,8
